In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers







In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
import pandas as pd
column_names=["Comment", "Label", "Source"]
df_a = pd.read_csv(r'C:\Users\dbehl\OneDrive\WGU\D213-Task 2\amazon_cells_labelled.txt', names= column_names, sep='\t', header = None)
df_a = df_a.assign(Source='Amazon')
df_i = pd.read_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\imdb_labelled.csv", sep =',', names=column_names, header = None)
df_i = df_i.assign(Source='IMDB')
df_y = pd.read_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\yelp_labelled.txt", sep = '\t', names=column_names, header = None)
df_y = df_y.assign(Source = 'Yelp')

print(df_a.shape)
print(df_i.shape)
print(df_y.shape)

In [ ]:
df = pd.concat([df_a, df_i, df_y], ignore_index=True)
print(df.shape)
pd.set_option('display.max_colwidth', 5000)
df.head(100)

In [ ]:

import seaborn as sns
df.Label = df.Label.astype(int)
df.info()
sns.countplot(df['Label'], hue =df['Source'], palette ='Set3')

#Data is split; 1,500 Positive and 1,500 Negative Reviews

In [ ]:
import advertools as adv
emoji_summary = adv.extract_emoji(df)
print(emoji_summary.keys())

In [ ]:
emoji_summary['emoji_counts']

In [ ]:
#B-1.A
pd.set_option('display.max_colwidth', 5000)
df.head(10)

In [ ]:
#Convert all text to lowercase

df['comment'] = df['Comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

In [ ]:
#remove all punctuation

df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

In [ ]:
#remove emoji's

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))


In [ ]:
#(Kosaka, 2020)

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df.head(20)



In [ ]:
#(SpaCy 101: Everything You Need to Know · SpaCy Usage Documentation, 2016)
#Lemmitization
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])
df['comment']= df['comment'].apply(space)
df.head(20)

In [ ]:
#Calculate vocabulary size
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['comment'])
print("Vocabulary size: ", len(tokenizer.word_index)+1)

In [ ]:
#B-1.C
#Word Embedding length is the fourth root of the vocabulary size (4445
max_sequence_embedding=int(round(np.sqrt(np.sqrt(4445))))
print("Word embedding length:") 
max_sequence_embedding

In [ ]:
#B-1.D
comment_length = []
for char_len in df['comment']:
    comment_length.append(len(char_len.split(' ')))
    
comment_max = np.max(comment_length)
comment_min = np.min(comment_length)
comment_median = np.median(comment_length)

print("The maximum length of our sequences would be:", comment_max)
print("The minimum length of our sequences would be:", comment_min)
print("The median length of our sequences would be:", comment_median)

max_length=41


In [ ]:
df= df.drop('Comment', axis =1)


In [ ]:
df=df.drop('Source', axis =1)
df.head(10)


In [ ]:
#split into train/test test
from sklearn.model_selection import train_test_split
x = df['comment']
y = df['Label']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state = 42)



In [ ]:
#B2 & B3

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding





#B-1.B
from keras.preprocessing.text import Tokenizer 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)
train_padded = pad_sequences(train_sequences, padding ='post', maxlen=max_length)
test_padded = pad_sequences(test_sequences, padding ='post', maxlen=max_length)

train_padded.shape

In [ ]:
print(word_index)

In [ ]:
train_padded

In [ ]:
np.asarray(y_train)
np.asarray(y_test)
np.asarray(x_test)
np.asarray(x_train)

In [ ]:
df_x_training = pd.DataFrame(train_padded)
df_x_testing = pd.DataFrame(test_padded)
df_y_training = pd.DataFrame(y_train)
df_y_testing = pd.DataFrame(y_test)

df_x_training.to_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\Padded_x_training.csv")
df_x_testing.to_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\Padded_x_testing.csv")
df_y_training.to_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\Label_train.csv")
df_y_testing.to_csv(r"C:\Users\dbehl\OneDrive\WGU\D213-Task 2\Label_test.csv")

In [ ]:


from keras.callbacks import EarlyStopping
activation = 'sigmoid'
loss = 'binary_crossentropy'
optimizer = 'adam'

num_epochs = 20

#define early_stoopping monitor
early_stopping_monitor = EarlyStopping(patience=2)

model = keras.Sequential(
    [
        layers.Embedding(vocab_size, max_sequence_embedding, input_length=max_length),
        layers.GlobalAveragePooling1D(),
        layers.Dense(100, activation="relu"),
        layers.Dense(50, activation="relu"),
        layers.Dense(1,activation=activation),
    ]
)

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model.summary()







In [ ]:
print(y_train.shape)

In [ ]:
print(train_padded.shape)

In [ ]:
type(y_train)
np.asarray(y_train)

In [ ]:
type(train_padded)

In [ ]:
history = model.fit(train_padded, y_train, epochs=num_epochs,
                  validation_split = 0.3, callbacks=[early_stopping_monitor], verbose = True)

In [ ]:
score=model.evaluate(test_padded, y_test, verbose=0)
print(f'Test loss: {score[0]}/Test accuracy: {score[1]}')

In [ ]:
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig("Accuracy plot.jpg")

In [ ]:
model.save('D213-Task 2- Sentiment Analysis.h5')